In [1]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(2019)

2022-11-21 15:56:07.108437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 15:56:08.199210: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-21 15:56:12.600967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-21 15:56:12.601039: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(7,activation = "softmax")   #Adding the Output Layer
])

2022-11-21 15:56:20.338794: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-21 15:56:20.338833: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (itish-HP-Pavilion-Gaming-Laptop-15-dk1xxx): /proc/driver/nvidia/version does not exist
2022-11-21 15:56:20.339378: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 178, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 20, 20, 64)       0

In [4]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(lr=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
bs=30         #Setting batch size
train_dir = "/home/itish/Downloads/voogle/train"   #Setting training directory
validation_dir = "/home/itish/Downloads/voogle/train"   #Setting testing directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(180,180))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180))

Found 168 images belonging to 7 classes.
Found 168 images belonging to 7 classes.


In [6]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=150 // bs,
                    epochs=10,
                    validation_steps=50 // bs,
                    verbose=2)

Epoch 1/10
5/5 - 4s - loss: 1.9361 - acc: 0.1377 - val_loss: 1.8542 - val_acc: 0.0667 - 4s/epoch - 779ms/step
Epoch 2/10
5/5 - 5s - loss: 1.7572 - acc: 0.3623 - val_loss: 1.1046 - val_acc: 0.8000 - 5s/epoch - 1s/step
Epoch 3/10
5/5 - 5s - loss: 1.1960 - acc: 0.5435 - val_loss: 0.3699 - val_acc: 0.9667 - 5s/epoch - 982ms/step
Epoch 4/10
5/5 - 5s - loss: 0.6224 - acc: 0.8261 - val_loss: 0.0510 - val_acc: 1.0000 - 5s/epoch - 1s/step
Epoch 5/10
5/5 - 4s - loss: 0.1314 - acc: 0.9565 - val_loss: 0.0022 - val_acc: 1.0000 - 4s/epoch - 891ms/step
Epoch 6/10
5/5 - 2s - loss: 0.0834 - acc: 0.9710 - val_loss: 0.0026 - val_acc: 1.0000 - 2s/epoch - 456ms/step
Epoch 7/10
5/5 - 2s - loss: 0.0243 - acc: 0.9933 - val_loss: 0.0070 - val_acc: 1.0000 - 2s/epoch - 488ms/step
Epoch 8/10
5/5 - 2s - loss: 0.0267 - acc: 0.9928 - val_loss: 3.2918e-05 - val_acc: 1.0000 - 2s/epoch - 476ms/step
Epoch 9/10
5/5 - 4s - loss: 0.0044 - acc: 1.0000 - val_loss: 3.0300e-05 - val_acc: 1.0000 - 4s/epoch - 825ms/step
Epoch 10

In [7]:
model.save("skyfall.h5")
print("Saved model to disk")

Saved model to disk


In [9]:
file1 = open('labels.txt', 'r')
Lines = file1.read().splitlines()
classes = []
for l in Lines:
    classes.append(l)

In [11]:
#to predict new images 
def predict_image(imagepath, classifier):
    predict = load_img(imagepath, target_size = (180, 180))   
    predict_modified = img_to_array(predict)
    predict_modified = predict_modified / 255
    predict_modified = np.expand_dims(predict_modified, axis = 0)
    result = classifier.predict(predict_modified)
    MaxPosition=np.argmax(result)  
    prediction_label=classes[MaxPosition]
    return prediction_label

In [12]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
predict_image('/home/itish/Downloads/voogle/test.jpg',model)

1/1 [==============================] - 0s 105ms/step


'Skyfall__00:02:20_bottle'